In [1]:
import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests
from matplotlib import patches

import os
import requests
import matplotlib.pyplot as plt

from PIL import Image, ImageDraw

from json import dump

from dotenv import load_dotenv

import cv2

In [2]:
load_dotenv()

cog_endpoint = os.getenv("COG_SERVICE_ENDPOINT") + "/face/v1.0/detect"
cog_key = os.getenv("COG_SERVICE_KEY")

In [3]:
key = cv2. waitKey(1)
webcam = cv2.VideoCapture(0)
while True:
    try:
        check, frame = webcam.read()
        cv2.imshow("Capturing", frame)
        key = cv2.waitKey(1)
        if key == ord('s'):
            
            ### Entrez dans filename le nom du fichier .jpg correspondant à l'émotion que vous simulez (par exemple : happy.jpg) ###
            cv2.imwrite(filename='normal.jpg', img=frame)
            webcam.release()
            cv2.waitKey(1650)
            cv2.destroyAllWindows()
            print("Processing image...")
            
            print("Image saved!")
        
            break
        elif key == ord('q'):
            print("Turning off camera.")
            webcam.release()
            print("Camera off.")
            print("Program ended.")
            cv2.destroyAllWindows()
            break
        
    except(KeyboardInterrupt):
        print("Turning off camera.")
        webcam.release()
        print("Camera off.")
        print("Program ended.")
        cv2.destroyAllWindows()
        break

Processing image...
Image saved!


In [4]:
# Load Image
img = './normal.jpg'
pil_image = Image.open(img, 'r')

# Show Image
plt.figure(figsize=(10, 5))
plt.imshow(pil_image)
plt.show()

<Figure size 720x360 with 1 Axes>

In [5]:
headers = {
    'Ocp-Apim-Subscription-Key': cog_key,
    'Content-Type': 'application/octet-stream'
}

params = {
    'returnFaceId': 'false',
    'returnFaceLandmarks': 'true',
    'returnFaceAttributes': 'age,gender,glasses,smile'
}

image_data = open(img, 'rb').read()



response = requests.post(
    cog_endpoint,
    params=params,
    headers=headers,
    data=image_data
)

In [6]:
response.status_code

200

In [7]:
response_data = response.json()
pprint.pprint(response_data)

[{'faceAttributes': {'age': 21.0,
                     'gender': 'male',
                     'glasses': 'NoGlasses',
                     'smile': 0.0},
  'faceLandmarks': {'eyeLeftBottom': {'x': 302.1, 'y': 154.7},
                    'eyeLeftInner': {'x': 315.0, 'y': 149.3},
                    'eyeLeftOuter': {'x': 288.1, 'y': 151.0},
                    'eyeLeftTop': {'x': 301.8, 'y': 144.0},
                    'eyeRightBottom': {'x': 385.4, 'y': 151.3},
                    'eyeRightInner': {'x': 370.3, 'y': 148.5},
                    'eyeRightOuter': {'x': 397.9, 'y': 145.8},
                    'eyeRightTop': {'x': 383.7, 'y': 140.8},
                    'eyebrowLeftInner': {'x': 323.7, 'y': 131.0},
                    'eyebrowLeftOuter': {'x': 266.8, 'y': 139.6},
                    'eyebrowRightInner': {'x': 360.4, 'y': 130.4},
                    'eyebrowRightOuter': {'x': 413.9, 'y': 128.4},
                    'mouthLeft': {'x': 314.6, 'y': 238.3},
                    'mo

In [8]:
plt.figure(figsize=(15, 7))
plt.imshow(pil_image)
for detected_face in response_data:
    for landmark in detected_face['faceLandmarks'].values():
        plt.scatter([landmark['x']], [landmark['y']], color='limegreen', s=2)
plt.show()

<Figure size 1080x504 with 1 Axes>

In [10]:
def call_face_api(image_filepath):
    params = {
        'returnFaceId': 'false',
        'returnFaceLandmarks': 'true',
        'returnFaceAttributes': 'mask',
        'detectionModel': 'detection_03'
    }

    image_data = open(image_filepath, 'rb').read()

    response = requests.post(
        cog_endpoint,
        params=params,
        headers=headers,
        data=image_data
    )
    response_data = response.json()
    return response_data


def plot_image_with_mask_label(image_filepath):
    
    response_data = call_face_api(image_filepath)
    pil_image = Image.open(image_filepath, 'r')
    plt.figure(figsize=(15, 7))
    plt.imshow(pil_image)
    
    for detected_face in response_data:
        rectangle_data =  detected_face['faceRectangle']
        x = rectangle_data['left']
        y = rectangle_data['top']
        w = rectangle_data['width']
        h = rectangle_data['height']
        if detected_face['faceAttributes']['mask']['type'] == 'noMask':
            label_str = "No Mask"
            color_str = "red"
        else:
            label_str = "Wearing Mask"
            color_str = "limegreen"
        rect = patches.Rectangle(
            [x, y], w, h,
            linewidth=2, edgecolor=color_str, facecolor='none'
        )
        ax = plt.gca()
        plt.text(
            x,
            y + h + 60,
            label_str,
            size=15,
            c=color_str
        )
    print(detected_face)
    ax.add_patch(rect)

In [11]:
plot_image_with_mask_label('./normal.jpg')

{'faceRectangle': {'top': 59, 'left': 257, 'width': 170, 'height': 226}, 'faceLandmarks': {'pupilLeft': {'x': 302.3, 'y': 151.2}, 'pupilRight': {'x': 385.1, 'y': 147.4}, 'noseTip': {'x': 344.0, 'y': 188.3}, 'mouthLeft': {'x': 316.4, 'y': 236.5}, 'mouthRight': {'x': 374.4, 'y': 234.4}, 'eyebrowLeftOuter': {'x': 278.8, 'y': 132.1}, 'eyebrowLeftInner': {'x': 320.4, 'y': 130.1}, 'eyeLeftOuter': {'x': 287.4, 'y': 152.7}, 'eyeLeftTop': {'x': 303.7, 'y': 145.5}, 'eyeLeftBottom': {'x': 300.5, 'y': 155.4}, 'eyeLeftInner': {'x': 317.5, 'y': 151.1}, 'eyebrowRightInner': {'x': 362.3, 'y': 129.9}, 'eyebrowRightOuter': {'x': 408.8, 'y': 130.6}, 'eyeRightInner': {'x': 370.6, 'y': 149.2}, 'eyeRightTop': {'x': 383.1, 'y': 141.5}, 'eyeRightBottom': {'x': 386.4, 'y': 151.6}, 'eyeRightOuter': {'x': 400.4, 'y': 147.3}, 'noseRootLeft': {'x': 330.4, 'y': 154.5}, 'noseRootRight': {'x': 355.5, 'y': 152.6}, 'noseLeftAlarTop': {'x': 325.8, 'y': 180.6}, 'noseRightAlarTop': {'x': 361.7, 'y': 178.7}, 'noseLeftAlarO

<Figure size 1080x504 with 1 Axes>

In [12]:
lst = []

def plot_image_with_mask_label(image_filepath):
    
    response_data = call_face_api(image_filepath)
    
    for detected_face in response_data:
        if detected_face['faceAttributes']['mask']['type'] == 'noMask':
            lst.append(0)
        else:
            lst.append(1)


In [13]:
plot_image_with_mask_label('./normal.jpg')

In [14]:
df = pd.read_csv("all_mask.csv")

In [15]:
arr_img = []

for path in df['image path']:
        
    img = Image.open(path).convert("L")
    pix = np.reshape(np.array(img, dtype='uint16'),(224*224))
    arr_img.append(pix)

In [32]:
df_test = df.copy()

In [33]:
df_test['values'].value_counts()

0    310
1    310
Name: values, dtype: int64

In [34]:
import time

In [35]:
lst = []

def plot_image_with_mask_label(image_filepath):
    
    response_data = call_face_api(image_filepath)
    
    for detected_face in response_data:
        time.sleep(5) 
        if detected_face['faceAttributes']['mask']['type'] == 'noMask':
            lst.append(0)
        else:
            lst.append(1)

for path in df_test['image path']:
    plot_image_with_mask_label(path)

In [36]:
len(lst)

620

In [37]:
from sklearn.metrics import classification_report

print(classification_report(df_test['values'], lst))

              precision    recall  f1-score   support

           0       1.00      0.93      0.96       310
           1       0.93      1.00      0.96       310

    accuracy                           0.96       620
   macro avg       0.96      0.96      0.96       620
weighted avg       0.96      0.96      0.96       620



In [38]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import seaborn as sns

cm = confusion_matrix(df_test['values'], lst)
sns.heatmap(cm/np.sum(cm), annot=True, fmt='.2%')

<AxesSubplot:>

<Figure size 432x288 with 2 Axes>